# Import & Load Election results data

### Config

In [ ]:
import sys
import configparser
config = configparser.ConfigParser()
config.read_file(open('../../../settings.ini'))

sys.path.insert(0, config.get('PATHS','libs_path'))
engine_path = config.get('DATABASE','engine_path')

### Main params

In [ ]:
remote_path = 'http://pacha.datawheel.us/datachile/polls/04_todos/'
local_path = '../data/'

### Imports

In [ ]:
import postgres #from local file postgres.py
from commons import inline_table_xml, download_file #from local file commons.py

import json
import pandas as pd
from sqlalchemy import create_engine

### Load file

In [ ]:
df = download_file(remote_path,local_path,'resultados_votaciones.csv')

#df = df[['year','election_type','comuna_datachile_id','partido_id','mesa_id','candidato_id','votos']]
df = df.rename(columns={'votos':'votes','candidato_id':'candidate_id','partido_id':'party_id','distrito_id':'district_id','election_type':'election_id','circunscripcion_id':'circunscription_id','lugar':'place','electo':'elected'})

df[['region_id']] = df[['region_id']].fillna(value=-1)
df[['circunscription_id']] = df[['circunscription_id']].fillna(value=-1)
df[['district_id']] = df[['district_id']].fillna(value=-1)
df[['comuna_datachile_id']] = df[['comuna_datachile_id']].fillna(value=-1)

df = df.astype({'comuna_datachile_id':'int', 'votes':'int','party_id':'int','place':'int','election_id':'int','year':'int','candidate_id':'int','district_id':'int','circunscription_id':'int','region_id':'int'})

list(df)

### Ingest

In [ ]:
engine = create_engine(engine_path)
db = postgres.PostgresDriver(engine)
db.to_sql(df, 'politics', 'fact_election_results')

### Update empty values

In [ ]:
engine.execute("""
UPDATE politics.fact_election_results SET region_id = NULL where region_id = -1;
""")

engine.execute("""
UPDATE politics.fact_election_results SET circunscription_id = NULL where circunscription_id = -1;
""")

engine.execute("""
UPDATE politics.fact_election_results SET district_id = NULL where district_id = -1;
""")

engine.execute("""
UPDATE politics.fact_election_results SET comuna_datachile_id = NULL where comuna_datachile_id = -1;
""")

### Indexes & FK

In [ ]:
engine.execute("""
CREATE INDEX fact_election_results_comuna_datachile_id 
ON politics.fact_election_results (comuna_datachile_id)
""")

engine.execute("""
CREATE INDEX fact_election_results_region_id 
ON politics.fact_election_results (region_id)
""")

### Related dim

In [ ]:
d1 = download_file(remote_path,local_path,'election_id.csv')
print (inline_table_xml(d1, 'election', 'election_id', 'election'))

In [ ]:
d2 = download_file(remote_path,local_path,'partidos_id.csv')
print (inline_table_xml(d2, 'partido', 'partido_id', 'partido'))